<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<p><p><p><p>
<b>Rubin Image Cutout Service Tutorial</b> <br>
Contact author: <i>Leanne Guy</i> <br>
Last verified to run: <i>2022-04-29</i> <br>
LSST Science Piplines version: Weekly <i>2022_17</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>intermediate</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [ ]:
# %load_ext pycodestyle_magic
# %flake8_on
# import logging
# logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

This notebook provides: 
1. An introduction to the IVOA ObsCore table and its use


**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:** This tutorial was developed for DP0.2 by Leanne Guy. Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction (TODO)

This Jupyter Notebook will illustrate how to use the Rubin Image Cutout Service

This service is fully SODA compliant and the SIAv2 interface is the way to access the service and get images. Add in the IVOA spec documents

The API read-the-docs is: https://pyvo.readthedocs.io/en/latest/_modules/pyvo/dal/sia2.html

A useful unit test: https://github.com/lsst-dm/image_cutout_backend/blob/main/tests/test_imageCutoutsBackend.py

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._
This notebook will teach the user how to use the LSST Image Cutout Service.

_Cite or link to any external information or documentation, and cross-reference to other notebooks._
*  https://dmtn-208.lsst.io/
*  https://dmtn-139.lsst.io

https://data-int.lsst.cloud/api/cutout/sync?id=8a953c0321bd4878bfa694dbf628ea81&circle=53.13925%20-34.0215%200.0105


Sme goals
* Explore the ivoa.ObsCore table and understand its use. 

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [ ]:
# Import general python packages
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal
import uuid
import requests

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query
import lsst.daf.butler as Butler
import lsst.geom
import lsst.resources
import lsst.geom as geom

import pyvo
from lsst.afw.image.exposure import Exposure, ExposureF
from lsst.afw.image import ImageF
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Plotting with MPL
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage

afwDisplay.setDefaultBackend('matplotlib')

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_access_token

# Ignore warnings
import warnings
from astropy.units import UnitsWarning
from astropy.io import fits

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.visualization import  ZScaleInterval, AsinhStretch
from astropy.wcs import WCS          # imports astropy's World Coordinate System function WCS
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize,  ZScaleInterval
from astropy.visualization.stretch import SinhStretch, LinearStretch, SqrtStretch

from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Holoviz for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

In [ ]:
# Ignore warnings
warnings.simplefilter("ignore", category=UnitsWarning)
warnings.simplefilter("ignore", category=UserWarning)

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

# Configure bokeh to generate output in notebook cells when show() is called.
output_notebook()

In [ ]:
# This should match the verified version listed at the start of the notebook
! echo ${IMAGE_DESCRIPTION}
! eups list lsst_distrib

### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

In [ ]:
# Function to autogenerate a plot title from the dataId.
def dataIdToString(dataId: dict) -> str:
    """Run the finder on the given dataset with the given butler.
   
   Parameters
    ----------
    ref : `DataId`
        the dataId 
   
   Returns
    -------
    title : `str` (only if result is not `None`)
        Plot title from string
    """
    
    title = "DC2 image: "
    for key, value in dataId.items():
        title += str(key) + ": " + str(value) + " "
    return title.strip() 

In [ ]:
def transformAndPlotImage(image: Exposure, img_opts: dict = None):
    """Plot and image using holoviews
   
   Parameters
    ----------
    image : `Exposure`
        the image to plot
        
    opts : `
   
   Returns
    -------
    title : `str` (only if result is not `None`)
        Plot title from string
    """
    arrayData = image.image.array
    
    # Apply a asinh/zscale mapping to the data    
    transform = ZScaleInterval() + AsinhStretch()
    
    scaledArrayData = transform(arrayData)
    scaledFlippedArrayData = np.flipud(scaledArrayData)    
    bbox_img = coadd_calexp.getBBox()
    bounds_img = (bbox_img.endX, bbox_img.endY,bbox_img.beginX, bbox_img.beginY)
    
    # Define some default plot options for the Image if not passed
    if img_opts is None:
        img_opts = dict(height=600, width=700,  xaxis="bottom", 
                        padding = 0.01, fontsize={'title': '8pt'},
                        colorbar=True, toolbar='right', show_grid=True,
                        tools=['hover']
                   )
    # Overlay the WCS
    wcs = image.getWcs()
    projection=WCS(image.getWcs().getFitsMetadata())
    
    # Create the Image element.
    img = hv.Image(scaledFlippedArrayData, 
                   bounds=bounds_img, 
                   kdims=['x', 'y']).opts(
        cmap = "Gray",  xlabel = 'X', ylabel ='Y',  title = "Image Cutout Demo", **img_opts)
    # img = img.redim.range(z=(0.3, 0.8))
    return rasterize(img)

## 2. Data Preparation


In [ ]:
# Butler repositoty 
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler.Butler(config, collections=collection)
registry = butler.registry

In [ ]:
# Let's use the same image as NB 3
dataId_coadd = dict(tract=4431, patch=17, band="i", skymap="DC2") 
datasetRef_coadd = registry.findDataset('deepCoadd_calexp', dataId_coadd)
coadd_calexp = butler.getDirect(datasetRef_coadd)
assert type(coadd_calexp) == lsst.afw.image.exposure.ExposureF
print(coadd_calexp.getFilter())

In [ ]:
# PVI 
dataId_calexp = {'visit': 192350, 'detector': 175, 'band': 'i'}
datasetRef_calexp = registry.findDataset('calexp', dataId_calexp)
calexp = butler.getDirect(datasetRef_calexp )
ci = calexp.getInfo().getId()

We are going to need the DataSetRef so we are going to use a differnt way to get the Butler info
We can get first the dataset reference from the butler knowing the dataid. butler.get and butler.getDirect can be used to then get the exposure but butler.get will take teh datasetRef and convert it back to a dataId and datasetType , query the registry again to make sure everything is consistent. getDirect bypasses this and assuems that the datasetRef I have   was gotten fro the registry, and is hence much more efficient. 

In [ ]:
# Get the WCS 
wcs = coadd_calexp.getWcs()
projection=WCS(coadd_calexp.getWcs().getFitsMetadata())

In [ ]:
datasetRef_coadd.id

In [ ]:
# Plot the image + mask + maskedimage
# TODO 
bbox_img = coadd_calexp.getBBox()
bbox_img.endX

In [ ]:
# Visualze the image  
# Look at the rich galaxy cluster in the botton left!!!
transformAndPlotImage(coadd_calexp)

In [ ]:
# Plot the same image but with matlotlib
import matplotlib.pyplot as plt      # imports matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

# create a matplotlib.pyplot figure
fig, ax = plt.subplots()
# get an alias to the lsst.afw.display.Display() method
display = afwDisplay.Display(frame=fig)
# set the image stretch algorithm and range
display.scale('asinh', 'zscale')
# load the image into the display
display.mtv(coadd_calexp.image)
plt.show()

## 3.0 Get a cutout of the galaxy cluster 

Let's get a cutout of the galaxy cluster in the bottom left
The cluster seems to be centered at about (X, Y) = (12500, 8500).
We can use the "pixelToSky" method of the WCS to get the sky coordinates:

Need the Butler UUID of an image 

The initial implementation of the image cutout service will only return FITS files

Initial implementation supports 
CIRCLE and POLYGON.  POS=RANGE not implemented - check!

The initial version of the cutout service will only support a single ID parameter and a single stencil parameter.
The ID parameter must be a UUID assigned by the Butler and uniquely identifying a source image

### 3.1 Use the Butler to get an image cutout

In [ ]:
# Let's get teh coordinates of the gallaxy cluster and define a circle around them radec = wcs.pixelToSky(12500, 8500)
# The cluster seems to be centered at about (X, Y) = (12500, 8500).
# We can use the "pixelToSky" method of the WCS to get the sky coordinates:
x,y = (12560, 8550)
radec = wcs.pixelToSky(x,y)
print(radec.getRa().asDegrees(), radec.getDec().asDegrees())

**TODO** - automate the extraction of x,y, using holoviz/streams tooling to interactively find the point of interest

In [ ]:
# Defin a circle around the point
point = lsst.geom.SpherePoint(radec.getRa().asDegrees() * lsst.geom.degrees, radec.getDec().asDegrees() * lsst.geom.degrees)
radius = 10 * lsst.geom.arcseconds
point, radius

In [ ]:
# Need the UUID for the cutout service 
calexp_uuid = datasetRef_coadd.id
assert isinstance(calexp_uuid, uuid.UUID)

In [ ]:
# Use the Butler to get the cutout
cutoutSideLength = 1000
cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
xy = geom.PointI(x,y)
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
parameters = {'bbox': bbox}
dsType = "deepCoadd"

In [ ]:
cutout_image = butler.get(dsType, parameters=parameters, dataId=dataId_coadd)
assert cutout_image is not None
print("The size of the cutout in pixels is: ", cutout_image.image.array.shape)

In [ ]:
transformAndPlotImage(cutout_image)

In [ ]:
# Show how to overlay the WCS and extract the centre of the galaxy cluster
wcs = coadd_calexp.getWcs()
print(wcs)
radec = wcs.pixelToSky(12500, 8500)
ra, dec = radec.getRa().asDegrees(), radec.getDec().asDegrees()
print(ra, dec)

### 3.2 Understanding the ivoa ObsCore table and its use

Add in here a description of what the ObsCore service is. 

The IVOA-defined obscore table contains generic metadata for datasets held at the IDF. The table is accessible via ADQL queries via a TAP endpoint.


In this example, we query the 

In [ ]:
service = get_tap_service()

In [ ]:
query = "SELECT COUNT(*) from ivoa.ObsCore"
result = service.search(query).to_table()
result

The ivoa.ObsCore contains 8475974 entries

In [ ]:
# Let's look at the exact calexp from above 
# First recall the dataId 
print(dataId_calexp)

In [ ]:
lsst_tract = dataId_calexp.get('tract')
lsst_patch = dataId_calexp.get('patch')
lsst_band = dataId_calexp.get('band')

In [ ]:
# The image UUID is part of the access url field
# TODO- is there an easier more intuitive way to access the URL?
url_str = '%' + str(datasetRef_coadd.id)
print(url_str)

In [ ]:
# Build the query 
# THIS DID WORK????? it now returns no results
query = """SELECT * FROM ivoa.ObsCore  
WHERE obs_collection = 'LSST.DP02' AND dataproduct_type = 'image' 
AND instrument_name = 'LSSTCam-imSim' 
AND access_url like '""" + url_str + """'
"""
print(query)

In [ ]:
# Build a query with no selection on UUID 
query = """SELECT * FROM ivoa.ObsCore  
WHERE obs_collection = 'LSST.DP02' AND dataproduct_type = 'image' 
AND instrument_name = 'LSSTCam-imSim' 
LIMIT 5
"""
print(query)

In [ ]:
results = service.search(query).to_table()

In [ ]:
# We passed a single UUID (in the first query) so there must be 1 result only (but there are zero)
# assert len(results) == 1  
print(len(results))
results

In [ ]:
# Select all images in a region
query = """SELECT * FROM ivoa.ObsCore 
WHERE obs_collection = 'LSST.DP02' AND dataproduct_type = 'image' 
AND instrument_name = 'LSSTCam-imSim' 
AND CONTAINS(POINT('ICRS', 62.61, -36.57), s_region)=1 
"""
print(query)

In [ ]:
# limit to 5 images for initial testing 
result = service.search(query, maxrec=5).to_table()
result

In [ ]:
# Get our RSP access token (we will need this to download the data)
token = get_access_token()

In [ ]:
# Let's take a qick look at the ivoa.ObsCore schema
# Prepare the query to explore the tables in the DP0.1 schema
query = "SELECT * FROM tap_schema.tables where schema_name like 'ivoa' order by table_index ASC"
print(query)

In [ ]:
results = service.search(query).to_table()
results

In [ ]:
# Return the first calep found in the ObsCore table 
#aurl = "https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/20d28216-534a-4102-b8a7-1c7f32a9b78c"
query = """
SELECT * FROM ivoa.ObsCore WHERE access_url like '%20d28216-534a-4102-b8a7-1c7f32a9b78c' 
"""
print(query)

In [ ]:
results = service.search(query)
r = results
results.to_table().show_in_notebook()

### 3.3 Use the Image Cutout Service with PyVO
Examples: 
https://github.com/astropy/pyvo/blob/main/examples/images/ex_get_cutouts.py
https://github.com/astropy/pyvo/blob/main/examples/images/ex_casA_image_cat.py

In [ ]:
# PyVo : # https://pyvo.readthedocs.io/en/latest/api/pyvo.dal.SIAService.html
# TODO provide an introduction to SIA/SODA in the description section
from pyvo.dal.sia import search, SIAService
import pyvo as vo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

In [ ]:
# We need to pass an authenticated session to the image cutout service. We will get this from the TAP service 
service = lsst.rsp.get_tap_service()
assert service is not None
assert service.baseurl == 'https://data.lsst.cloud/api/tap'

In [ ]:
# Extract the session authentiction for reuse
auth_session = service._session

In [ ]:
# Now lets retrieve the first calibrated exposure 
results = service.search("SELECT * FROM ivoa.ObsCore WHERE dataproduct_subtype='lsst.calexp' LIMIT 1")

In [ ]:
# Or let's retrieve the image with a nice galaxy cluster above
results = service.search("SELECT * FROM ivoa.ObsCore WHERE dataproduct_subtype='lsst.calexp' LIMIT 1")

In [ ]:
# The result returned by the TAP service is not the image itself but rather an access URL for the image
# NOTE : here a lot more description of the datalinks service shoudl be added
results.to_table().show_in_notebook()

In [ ]:
# Extract the access URL from the result in the first row
result = results[0]
f"Datalink link service url: {result.getdataurl()}"

In [ ]:
# Attempt to set up auth following the recipe in get_tap_service()
# def setup_auth():
#     s = requests.Session()
#     tok = get_access_token()
#     assert tok is not None

#     s.headers["Authorization"] = "Bearer " + tok
#     auth = pyvo.auth.authsession.AuthSession()
#     auth.credentials.set("lsst-token", s)
    
#     datalink_url = os.getenv("EXTERNAL_INSTANCE_URL", "") + "/api/datalink"
#     cutout_url = os.getenv("EXTERNAL_INSTANCE_URL", "") + "/api/cutout"
    
#     auth.add_security_method_for_url(datalink_url, "lsst-token")
#     auth.add_security_method_for_url(datalink_url + "/links", "lsst-token")
#     auth.add_security_method_for_url(cutout_url, "lsst-token")
    
#     return auth

In [ ]:
# Document this better
#auth = setup_auth()
# s = requests.Session()
# tok = get_access_token()
# assert tok is not None

# s.headers["Authorization"] = "Bearer " + tok
# auth = pyvo.auth.authsession.AuthSession()
# auth.credentials.set("lsst-token", s)

In [ ]:
datalink_url = os.getenv("EXTERNAL_INSTANCE_URL", "") + "/api/datalink"
cutout_url = os.getenv("EXTERNAL_INSTANCE_URL", "") + "/api/cutout"

In [ ]:
# auth.add_security_method_for_url(datalink_url, "lsst-token")
# auth.add_security_method_for_url(datalink_url + "/links", "lsst-token")
# auth.add_security_method_for_url(cutout_url, "lsst-token")

In [ ]:
dl_results = DatalinkResults.from_result_url(result.getdataurl(),session=auth_session)
dl_results.to_table().show_in_notebook()

In [ ]:
print(dl_results.status)

In [ ]:
print(dl_results.votable)

In [ ]:
# Now we can grab the google signed URL for the image (note that this will expire)
image_url = dl_results.getrecord(0).get('access_url')
print(image_url)

In [ ]:
# Now let's download the image 
filename = download_file(image_url)
hdulist = fits.open(image_url)

In [ ]:
# Looks like a deepCoadd_calexp to me...
for hdu in hdulist:
    print(hdu.name)

In [ ]:
# Let's plot the image and see what it looks like...
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', interval=ZScaleInterval(), stretch=SqrtStretch(), cmap='gray')
fig.colorbar(im[0])

In [ ]:
# We can also create an ExposureF stack object - if say we want to do some reprocessing 
new_coadd = ExposureF(filename)

fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(new_coadd.maskedImage)
plt.show()

In [ ]:
# Now we can create the datalink, which will allow us to access the (transient) signed URL for image access
datalink = DatalinkResults.from_result_url(result.getdataurl(),session=auth_session)
print(datalink.status)
#print(datalink.votable)

# Now we can grab the google signed URL for the image (note that this will expire)
image_url = datalink.getrecord(0).get('access_url')
print(image_url)

In [ ]:
# Now let's retrive the image 

In [ ]:
# Gregory How do I call the cutout service to give me the same cutout as I get above from the Butler? 
calexp_uuid = datasetRef_calexp.id
cutout_coords = SkyCoord(radec.getRa().asDegrees()*u.degree, radec.getDec().asDegrees()*u.degree, unit="deg", frame="icrs")
cutout_radius = 10
cutout_pos = 'CIRCLE 55.8 -32.3 10.0'
cutout_pos

In [ ]:
help(service.search)

In [ ]:
# submit a SIA query to this service with the given parameters
# error : file does not appear to be a VOTable
# service.search(pos=cutout_coords)

In [ ]:
# fails
# service.search(pos=(350.85, 58.815),size=0.25,format="image/fits", )

In [ ]:
# fails: DALFormatError: E19: None:8:0: E19: File does not appear to be a VOTABLE

#cutouts1 = service.search(pos=(148.8888, 69.065), size=0.2)
#query1 = service.create_query(size=0.2)  # or create a query object
#query1.pos = (350.85, 58.815)
# cutouts2 = query1.execute()

In [ ]:
# fails: DALFormatError: E19: None:8:0: E19: File does not appear to be a VOTABLE
query3 = service.create_query(pos=(350.85, 58.815),
                              size=0.25, format="image/fits")
# images = query3.execute()
# firstim = images[0]

# Use SODA to create a cutout

In this section we make use of pyvo to call obstap, gets the results and then follows the links returned to get a cutout.

TODO: Describe te SODA service

Only synchronouse queries of the cutout service run currently. 
An asynchronous quert support may be introduced in the future

SODA wants the Butler UUID. Specifically, the URL is similar to 
https://data-int.lsst.cloud/api/cutout/sync?id=8a953c0321bd4878bfa694dbf628ea81&circle=53.13925%20-34.0215%200.0105 
(the id parameter is the Butler UUID, and the remaining parameters are the cutout request, 
and there's a POST version as well and an async version following the SODA standard).

In [ ]:
# Let's use the same coadd as before 
url_str= '%' + str(datasetRef_coadd.id) + '%'
q = """
SELECT * FROM ivoa.ObsCore 
WHERE access_url like '""" + url_str + """'
""" 

In [ ]:
# Use a simple query to return the coadd with a nice galaxy cluster
# Note that the setup_auth() method needs to have been run
results = service.search(q, maxrec=1)
results.to_table().show_in_notebook()

In [ ]:
r=results[0]
print(f"Datalink link service url:  {r.getdataurl()}")
dr = DatalinkResults.from_result_url(r.getdataurl(), session=auth_session)
dr.to_table().show_in_notebook()

You will see above that there are 2 results. One is similar to above in that it provides an access URL to the image. The second is the cutout service. We will use this to get a cutout by using SodaQuery 

Note:  add some description and an introduction here as to what SODA and a SODA service is. 

In [ ]:
sq = SodaQuery.from_resource(dr, dr.get_adhocservice_by_id("cutout-sync"), 
                             session=auth_session)

# Now define a circle shape that provides an ra, dec and radius, that defines the shape of the cutout
# Describe the SodaQuery API here 

# Using the results 'r' above, we can see 
print(r["s_ra"], r["s_dec"])

In [ ]:
# rather than use the centre of the image, that we get from the query, we'll take the centre of the galaxy cluster, extracted above 
print(ra, dec)

In [ ]:
radius = 0.01 # units
#sq.circle = (r["s_ra"], r["s_dec"], radius)
sq.circle = (ra, dec, radius)

In [ ]:
print(type(sq))
print(sq.circle)
print(sq)

In [ ]:
sodaPoly = os.path.join(os.getenv('HOME'), 'DATA/soda-polygon.fits')
with open(sodaPoly, 'bw') as f:
    f.write(sq.execute_stream().read())

In [ ]:
# Display the cutout
i = ImageF(sodaPoly)     #read FITS file into afw image object
afw_display = lsst.afw.display.Display()      #get an alias to the lsst.afw.display.Display() method
afw_display.scale('asinh', 'zscale')    #set the image stretch algorithm and range
afw_display.mtv(i)                     #load the image into the display

In [ ]:
host=os.getenv("EXTERNAL_INSTANCE_URL")
print(host)
SODA_URL="{}/api/image/soda/sync".format(host)
ID='default.calexp.r'
print(SODA_URL)

In [ ]:
q3 = pyvo.dal.adhoc.SodaQuery(
    SODA_URL, 
    ID=ID, 
    POS='POLYGON 216.67 -0.52 216.685 -0.52 216.677 -0.54'
)
q3

In [ ]:
sodaPoly = os.path.join(os.getenv('HOME'), 'DATA/soda-polygon.fits')
with open(sodaPoly, 'bw+') as f3:
    f3.write(q3.execute_stream().read())

In [ ]:
# Some test objects
P0 = ['CIRCLE 1.0 2.0 3.0']
P1 = ['CIRCLE 1.0 2.0 3.0', 'RANGE 1.0 2.0 3.0 4.0']
service = SodaQuery(baseurl=SODA_URL)

In [ ]:
size = 20/3600 # 20 arcseconds
POS='CIRCLE 216.68 -0.53 %03.3f' % size

In [ ]:
POS